In [1]:
from __future__ import print_function
import numpy as np
import random as rnd
from sys import exit
from tqdm.autonotebook import tqdm
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


In [5]:
class Salesman:
    def __init__(self, c: int, a: int, start: int=0, alpha:float=.1, beta:float=1.1, gamma:float=.1, generate:bool=True, path:str="cities_example.txt", verbose:bool=False):
        self.cities = c
        self.start = start
        self.ants = a
        self.k = 10 * c
        
        self.verbose = verbose
        
        if generate:
            self.d = self.init_cities(2 * 5)
            np.savetxt(path, self.d, fmt="%i")
        else:
            try:
                self.d = np.loadtxt(path, dtype=int)
            except OSError:
                exit("Error: If you specify the path argument with generate=False, then you have to create a [path].txt file following cities_example.txt format. It should be a square matrix for distances between cities with zeroes on the diagonal.")
        self.p = np.zeros((c, c))
        self.c = np.zeros((self.k, c))
        # wtf
        self.q = 1
        self.gamma = gamma
        
        # wtf
        self.curiosity = 0.1
        self.alpha = alpha
        self.beta = beta
        
        if self.verbose:
            print("Initialized TSP with map:")
            print(self.d)
            print(f"Starting from town #{self.start}")
    
    
    def init_cities(self, maxd:int) -> np.ndarray:
        d = np.random.randint(-maxd, maxd, (self.cities, self.cities))
        d = np.abs((d + d.T) // 2) + 1
        np.fill_diagonal(d, 0)
        return d
    
    
    def update_pheromones(self, roads:list) -> None:
        # evaporation
        self.p *= .9
        for road in roads:
            d = sum([c[1] for c in road])
            s = self.start
            
            for c in road[1:]:
                self.p[s][c[0]] += self.q / d
                self.p[c[0]][s] += self.q / d
                s = c[0]
                
            if self.verbose:
                print(d, self.p)
        
        
    def run(self, it:int) -> tuple:
        minroad = ([], 1000000)
        for i in range(it):
            roads = [Ant(self.d, self.p, self.start, self.alpha, self.beta).walk() for _ in range(self.ants)]
            self.update_pheromones(roads)
            
            roadslength = [sum([c[1] for c in r]) for r in roads]
            roads = [[path[0] for path in r] for r in roads]
            bestroad = (np.argmin(roadslength), min(roadslength))
            
            minroad = (roads[bestroad[0]], bestroad[1]) if bestroad[1] < minroad[1] else minroad
            if self.verbose:
                print(*roads[bestroad[0]], bestroad[1])
        return minroad
    
    
class Ant:
    def __init__(self, c: np.ndarray, p: np.ndarray, s:int, a:float, b:float):
        self.cities = c
        self.p_map = p
        self.to_visit = [v for v in range(c.shape[0]) if not v == s]
        self.alpha = a
        self.beta = b
        
        self.road = [(s, 0)]
    
    def choose_road(self) -> int:
        crt = self.road[-1][0]
        ps = {}
        for c, _ in enumerate(self.to_visit):
            intensity = self.p_map[crt][c]
            desirability = 1 / self.cities[crt][c]
            ps[c] = intensity * self.alpha * desirability * self.beta
        pt = sum(ps.values())
        ps = {p: ps[p]/pt for p in ps}
        try:
            return np.random.choice(a=list(ps.keys()), p=list(ps.values()))
        except ValueError:
            return rnd.randint(0, len(self.to_visit) - 1)
    
    def walk(self) -> list:
        while len(self.to_visit) > 0:
            crt = self.road[-1][0]
            i = self.choose_road()
            c = self.to_visit.pop(i)
            self.road.append((c, self.cities[crt][c]))
        return self.road

In [6]:
# ants
# nb cities
# start (depending on nb cities)
# alpha
# beta
# iterations
# gamma
# curiosity
# ants = interact(lambda x: x, x=widgets.IntSlider(min=2, max=30, step=1, value=10, description="Ants count"))
# cities = interact(lambda x: x, x=widgets.IntSlider(min=1, max=30, step=1, value=10, description="Cities count"))
# start = interact(lambda x: x, x=widgets.IntSlider(min=0, max=30, step=1, value=10, description="Start from"))
# alpha = interact(lambda x: x, x=widgets.FloatSlider(min=0, max=1, step=.01, value=10, description="𝛼"))
# beta = interact(lambda x: x, x=widgets.FloatSlider(min=1, max=10, step=.01, value=10, description="𝛽"))
# iterations = interact(lambda x: x, x=widgets.IntSlider(min=1, max=1000, step=1, value=10, description="Iterations"))
# gamma = interact(lambda x: x, x=widgets.FloatSlider(min=-10, max=30, step=1, value=10, description="𝛾"))
# curiosity = interact(lambda x: x, x=widgets.FloatSlider(min=-10, max=30, step=1, value=10, description="Curiosity"))

benchmark = []
for i in tqdm(range(50)):
    salesman = Salesman(12, 50, start=0, generate=False, verbose=False)
    benchmark.append(salesman.run(100))
print(*benchmark, sep="\n")

/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:89: RuntimeWarning: divide by zero encountered in long_scalars
/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars
/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in double_scalars



([0, 2, 11, 4, 9, 6, 8, 7, 10, 5, 3, 1], 28)
([0, 2, 9, 8, 11, 4, 6, 5, 10, 7, 3, 1], 23)
([0, 6, 5, 2, 11, 8, 10, 7, 9, 4, 3, 1], 27)
([0, 2, 4, 9, 7, 10, 11, 8, 6, 5, 3, 1], 28)
([0, 2, 9, 5, 3, 1, 8, 10, 7, 6, 4, 11], 30)
([0, 2, 11, 10, 5, 6, 4, 3, 7, 9, 1, 8], 28)
([0, 1, 10, 7, 3, 5, 2, 11, 8, 6, 4, 9], 26)
([0, 2, 1, 3, 7, 9, 4, 11, 8, 6, 10, 5], 29)
([0, 2, 5, 6, 3, 7, 10, 9, 4, 11, 1, 8], 31)
([0, 2, 5, 6, 8, 11, 4, 3, 1, 9, 10, 7], 25)
([0, 1, 8, 11, 4, 7, 3, 5, 10, 6, 2, 9], 29)
([0, 1, 3, 7, 10, 8, 6, 4, 11, 2, 5, 9], 28)
([0, 1, 9, 7, 3, 5, 2, 6, 4, 11, 10, 8], 29)
([0, 2, 6, 3, 7, 10, 5, 8, 11, 4, 9, 1], 26)
([0, 2, 11, 8, 6, 4, 3, 5, 10, 1, 9, 7], 27)
([0, 2, 11, 6, 4, 9, 3, 7, 10, 5, 8, 1], 28)
([0, 2, 5, 10, 9, 7, 6, 4, 11, 8, 3, 1], 31)
([0, 6, 8, 1, 11, 2, 5, 10, 7, 9, 4, 3], 30)
([0, 2, 6, 8, 11, 4, 9, 5, 10, 1, 3, 7], 28)
([0, 2, 6, 5, 10, 7, 8, 11, 4, 9, 3, 1], 25)
([0, 1, 3, 9, 4, 11, 8, 7, 10, 5, 2, 6], 27)
([0, 2, 11, 4, 6, 5, 8, 10, 9, 7, 3, 1], 29)
([0, 2, 4

In [7]:
print(min([b[1] for b in benchmark]))


23
